# 🧠 Fine-Tuning LLMs with LoRA (Parameter-Efficient Fine-Tuning)

This notebook demonstrates how to fine-tune a **language model** using **Low-Rank Adaptation (LoRA)**.  
LoRA enables efficient adaptation of large models by updating only a small set of additional parameters rather than the full network.

---

## ⚙️ Workflow
1. Load a pre-trained model (`flan-t5-base`)
2. Attach LoRA adapters using 🤗 PEFT
3. Fine-tune on a small text dataset
4. Save & test the adapted model

---

## 🧩 Libraries Used
| Library | Purpose |
|----------|----------|
| `transformers` | Load model + tokenizer |
| `datasets` | Load training text |
| `peft` | Add LoRA layers |
| `accelerate` | Manage training |
| `bitsandbytes` | Quantized weights (8-bit) |
| `evaluate` | Metrics |

---

## 🎯 Objective
Show how to fine-tune an open-source LLM efficiently on Colab GPU with LoRA for tasks like text generation or QA.


In [ ]:
!pip install -q transformers datasets peft accelerate bitsandbytes evaluate sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.7 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("squad", split="train[:1%]")  # small subset for demo
dataset = dataset.map(lambda x: {"text": x["question"] + " " + x["context"]})
print(dataset[0]["text"])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

plain_text/validation-00000-of-00001.par(…):   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Map:   0%|          | 0/876 [00:00<?, ? examples/s]

To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map="auto", load_in_8bit=True)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
from torch.utils.data import DataLoader
import torch

def preprocess(batch):
    enc = tokenizer(batch["text"], truncation=True, padding="max_length", max_length=256)
    enc["labels"] = enc["input_ids"].copy()
    return enc

tokenized = dataset.map(preprocess, batched=True, remove_columns=dataset.column_names)
tokenized.set_format(type="torch")   # ✅ converts everything to tensors


Map:   0%|          | 0/876 [00:00<?, ? examples/s]

In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(tokenized, batch_size=2, shuffle=True)


In [ ]:
from torch.optim import AdamW
from tqdm import tqdm

optimizer = AdamW(model.parameters(), lr=1e-4)
num_epochs = 1
model.train()

for epoch in range(num_epochs):
    loop = tqdm(dataloader, desc=f"Epoch {epoch+1}")
    for batch in loop:
        batch = {k: v.to(model.device) for k, v in batch.items() if k in ["input_ids", "attention_mask", "labels"]}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        loop.set_postfix(loss=loss.item())


Epoch 1:   9%|▉         | 41/438 [3:29:52<33:54:03, 307.41s/it, loss=nan]